In [1]:
import numpy as np
from cohlib.utils import pickle_open
from cohlib.alg.laplace_sgc import SpikeTrial, TrialData

In [4]:
mu = -4.5
L = 25 
C = 30
sample_length = 1000
seed = 8

save_path = f'saved/synthetic_data/simple_synthetic_{L}_{sample_length}_{C}_{mu}_{seed}'

data_load = pickle_open(save_path)

data_load['meta'].keys()
Gamma = data_load['latent']['Gamma']
Wv = data_load['meta']['Wv']
freqs = data_load['meta']['freqs']
vs = data_load['latent']['vs']
zs = data_load['latent']['zs']
xs = data_load['latent']['xs']

spikes = data_load['observed']['spikes']
freqs = data_load['meta']['freqs']

In [ ]:
# sanity test on inverses... 
X_complex = Gamma[5,:,:]
X_complex_inv = np.linalg.inv(X_complex)


X_real = transform_cov_c2r(X_complex)
X_real_to_complex = transform_cov_r2c(X_real)
X_real_to_complex_inv = np.linalg.inv(X_real_to_complex)
# X_real



In [ ]:
np.isclose(X_complex_inv, X_real_to_complex_inv)

array([[ True,  True],
       [ True,  True]])

In [ ]:
X_real_to_complex_inv

array([[1.69349577-1.74767187e-19j, 0.14217476+4.78660442e-02j],
       [0.14217476-4.78660442e-02j, 2.71947932-3.93205500e-19j]])

In [ ]:
X_complex_inv

array([[1.69349577-1.74767187e-19j, 0.14217476+4.78660442e-02j],
       [0.14217476-4.78660442e-02j, 2.71947932-3.93205500e-19j]])

In [5]:
v[0] * (2*np.pi)/(sample_length/2)

-4.5

In [5]:
# sample_length = data_load['meta']['sample_length']
# fs = data_load['meta']['fs']
# frange = [0, int(fs/2)]
# res = fs/sample_length


num_J_vars = Wv.shape[1]
K = 2
Gamma_inv_init = np.eye(K*num_J_vars)

spikes_short = spikes[:10,:,:,:]
spikes_use = spikes_short
spikes_grouped = [spikes_use[:,:,k,:] for k in range(K)]

In [6]:
import numpy as np
import itertools
from cohlib.alg.laplace_sgc import TrialData, SpikeTrial
from scipy.linalg import block_diag
from cohlib.alg.em_sgc import get_trial_obj
from cohlib.alg.em_sgc import update_Gamma_complex_dc, construct_Gamma_full_real_dc
# Let's test out laplace approximation with Wv first
# trial = get_trial_obj(spikes_grouped, 0, Wv, Gamma_inv_init)

In [7]:
# mu, Ups_inv = trial.laplace_approx()
# mu = trial.laplace_approx()

In [7]:
# Gamma_est, prev_inv, track = fit_sgc_model(spikes_grouped, Wv, inits, num_em_iters=num_em, 
#             max_approx_iters=15, track=True)

inits = {'Gamma_inv_init': Gamma_inv_init}
num_em = 1

data = spikes_grouped
W = Wv
num_em_iters = num_em
max_approx_iters = 15
track = True

In [8]:
num_em_iters = 1
# safety / params
assert type(data) is list
K = len(data)

Ls = [data[i].shape[0] for i in range(K)]
L = Ls[0]
# assert all_equal(Ls)

num_timepts = W.shape[0]
num_J_vars = W.shape[1]

# inits
Gamma_inv_init = inits['Gamma_inv_init']

# alg
# TODO make classes NOT mutable
track_gamma = []
for r in range(num_em_iters):
    print(f'EM iter: {r}')
    if r == 0:
        Gamma_prev_inv = Gamma_inv_init

    mus = np.zeros((L,K*num_J_vars))
    Ups_invs = np.zeros((L,K*num_J_vars,K*num_J_vars))

    for l in range(L):
        print(f'Laplace Approx trial {l}')
        trial = get_trial_obj(data, l, W, Gamma_prev_inv)
        mu, Ups_inv = trial.laplace_approx()
        # real reprsentation
        mus[l,:] = mu
        Ups_invs[l,:,:] = Ups_inv


    # M-Step
    print(f'M-Step for EM iter {r}')
    DC_update, Gamma_update_complex = update_Gamma_complex_dc(mus, Ups_invs, K, num_J_vars) 
    DC_update = np.array([[1,0], [0,1]])
    Gamma_prev_inv = construct_Gamma_full_real_dc(DC_update, Gamma_update_complex, K, num_J_vars, invert=True)

EM iter: 0
Laplace Approx trial 0
Laplace Approx trial 1
Laplace Approx trial 2
Laplace Approx trial 3
Laplace Approx trial 4
Laplace Approx trial 5
Laplace Approx trial 6
Laplace Approx trial 7
Laplace Approx trial 8
Laplace Approx trial 9
M-Step for EM iter 0


In [17]:
np.where(np.diag(Ups_invs[3,:,:]) > 0)

(array([   0,  999, 1001, 2000]),)

In [22]:
from cohlib.alg.em_sgc import transform_cov_c2r, est_cov_r2c, rearrange_mat
from cohlib.alg.em_sgc import get_freq_vecs_real_dc
K = 2
dc = True
L = mus.shape[0]
if dc is True:
    J_nodc = int((num_J_vars-1)/2)
    J = J_nodc 
else:
    # J = int(num_J_vars/2)
    raise NotImplementedError

DC_mus_outer = np.zeros((L,K,K))
DC_Upss = np.zeros((L,K,K))
mus_outer = np.zeros((L,J,K*2,K*2))
Upss = np.zeros((L,J,K*2,K*2))

for l in range(L):
    Ups_inv_j_vecs = get_freq_vecs_real_dc(np.diag(Ups_invs[l,:,:]), K, num_J_vars)
    mu_js = get_freq_vecs_real_dc(mus[l,:], K,num_J_vars)

    DC_mus_outer[l,:,:] = np.outer(mu_js[0], mu_js[0])
    DC_Upss[l,:,:] = np.diag(1/Ups_inv_j_vecs[0])
    for j in range(J):
        mus_outer[l,j,:,:] = np.outer(mu_js[j+1], mu_js[j+1])
        Upss[l,j,:,:] = -np.diag(1/Ups_inv_j_vecs[j+1])

# enforce circulary symmetry
k_mask_pre = 1 - np.eye(2)
k_mask_inv = block_diag(*[k_mask_pre for k in range(K)])
k_mask =  1 - k_mask_inv

Gamma_update_real = (mus_outer*k_mask[None,None,:,:]).sum(0) + Upss.sum(0)
Gamma_update_real = Gamma_update_real / L



DC_update = np.zeros((K,K))
Gamma_update_complex = np.zeros((J,K,K), dtype=complex)
for l in range(L):
    DC = DC_mus_outer[l,:,:] + DC_Upss[l,:,:]
    Sig_real = mus_outer[l,:,:,:]*k_mask + Upss[l,:,:,:]
    Sig_complex = np.zeros((J,K,K), dtype=complex)

    for j in range(J):
        Sig_complex[j,:,:] = est_cov_r2c(rearrange_mat(Sig_real[j,:,:],K))

    DC_update += DC
    Gamma_update_complex += Sig_complex
DC_update = DC_update / L
Gamma_update_complex = Gamma_update_complex / L

In [106]:
Upss.shape

(10, 500, 4, 4)

In [105]:
def construct_Gamma_full_real_dc(DC_update, Gamma_update_real, K, num_J_vars, invert=False):
    J = int((num_J_vars-1)/2)


    Gamma_full = np.zeros((K*num_J_vars, K*num_J_vars))
    if invert == True:
        DC_update = np.linalg.inv(DC_update)
    base_filt = np.zeros(num_J_vars)
    base_filt[0] = 1
    j_filt = np.tile(base_filt.astype(bool), K)

    for k in range(K):
        Gamma_full[j_filt,k*num_J_vars] = DC_update[:,k]


    # Gamma_update_real_inv = np.stack([np.linalg.inv(Gamma_update_real[j]) for j in range(J)])

    for j in range(J):
        Gamma_j = Gamma_update_real[j,:,:]
        if invert == True:
            Gamma_j = np.linalg.inv(Gamma_j)
        # Gamma_n_real = reverse_rearrange_mat(transform_cov_c2r(Gamma_n),K)
        base_filt = np.zeros(num_J_vars)
        j_var = int(j*2 + 1)
        base_filt[j_var:j_var+2] = 1
        j_filt = np.tile(base_filt.astype(bool), K)
        # print(j_filt)
        for k in range(K):
            kj = int(k*2)
            Gamma_full[j_filt,k*num_J_vars+j_var:k*num_J_vars+j_var+2] = Gamma_j[:,kj:kj+2]

    return Gamma_full

In [76]:
from cohlib.alg.em_sgc import transform_cov_c2r, est_cov_r2c, reverse_rearrange_mat
transform_cov_r2c = est_cov_r2c
Gamma_update_complex_test = np.stack([transform_cov_r2c(rearrange_mat(Gamma_update_real[j,:,:], K)) for j in range(J)])
# Gamma_update_complex_test = np.stack([transform_cov_r2c(Gamma_update_real[j,:,:]) for j in range(J)])
np.all(np.isclose(Gamma_update_complex, Gamma_update_complex_test))

True

In [77]:
2*(10**-2)

0.02

In [78]:
import matplotlib.pyplot as plt
a = np.stack([transform_cov_r2c(np.linalg.inv(4*rearrange_mat(Gamma_update_real[j,:,:], K))) for j in range(J)])
# a = np.stack([np.linalg.inv(transform_cov_r2c(rearrange_mat(Gamma_update_real[j,:,:], K))) for j in range(J)])
b = np.stack([np.linalg.inv(Gamma_update_complex[j,:,:]) for j in range(J)])
j_ind = 20
print(np.round(a[j_ind], 4))
print(np.round(b[j_ind], 4))

[[ 0.0838-0.j -0.0001-0.j]
 [-0.0001+0.j  0.0826+0.j]]
[[ 0.0838+0.j -0.0001-0.j]
 [-0.0001+0.j  0.0826+0.j]]


In [102]:
Gamma_update_real_inv = np.stack([np.linalg.inv(Gamma_update_real[j]) for j in range(J)])
a = Gamma_update_real_inv
b = np.stack([reverse_rearrange_mat(transform_cov_c2r(np.linalg.inv(0.25*Gamma_update_complex[j,:,:])), K) for j in range(J)])

In [103]:
j_ind = 2
np.round(a[j_ind,:,:], 5)

array([[ 1.6731e-01,  0.0000e+00,  7.0000e-05, -1.2000e-04],
       [ 0.0000e+00,  1.6820e-01,  1.0000e-04, -1.6000e-04],
       [ 7.0000e-05,  1.0000e-04,  1.6541e-01, -0.0000e+00],
       [-1.2000e-04, -1.6000e-04, -0.0000e+00,  1.6522e-01]])

In [104]:
np.round(b[j_ind,:,:], 5)

array([[ 1.6775e-01,  0.0000e+00, -5.0000e-05, -1.1000e-04],
       [-0.0000e+00,  1.6775e-01,  1.1000e-04, -5.0000e-05],
       [-5.0000e-05,  1.1000e-04,  1.6531e-01,  0.0000e+00],
       [-1.1000e-04, -5.0000e-05, -0.0000e+00,  1.6531e-01]])

In [122]:
s2 = mus_outer.sum(0) + Upss.sum(0)

In [120]:
s = np.zeros((J, K*2, K*2))
for l in range(L):
   s += mus_outer[l,:,:,:] + Upss[l,:,:,:]

In [102]:
Sig_real[0,:,:]

array([[-5.67544657e+00,  0.00000000e+00,  6.50443706e-04,
        -1.25343308e-03],
       [ 0.00000000e+00, -5.64244188e+00, -3.01138207e-03,
         5.80306316e-03],
       [ 6.50443706e-04, -3.01138207e-03, -5.61728299e+00,
         0.00000000e+00],
       [-1.25343308e-03,  5.80306316e-03,  0.00000000e+00,
        -5.59357254e+00]])

In [116]:
Gamma_update_complex[9,:,:]

array([[0.78875217+0.j        , 1.954894  -2.30163536j],
       [1.954894  +2.30163536j, 1.53876173+0.j        ]])

In [202]:
from cohlib.alg.em_sgc import reverse_rearrange_mat, transform_cov_c2r
old = construct_Gamma_full_real(Gamma_update_complex, K, num_J_vars-1)
new = construct_Gamma_full_real_dc(DC_update, Gamma_update_complex, K, num_J_vars)

In [206]:
k = 1
new[j_filt,k*num_J_vars]

array([6928.57217625, 6912.43608406])

In [195]:
Gamma_full = np.zeros((K*num_J_vars, K*num_J_vars))
# if invert == True:
#     DC_update = np.linalg.inv(DC_update)
base_filt = np.zeros(num_J_vars)
base_filt[0] = 1
j_filt = np.tile(base_filt.astype(bool), K)

In [200]:
k = 0
Gamma_full[j_filt,k*num_J_vars]

array([0., 0.])

In [182]:
np.linalg.inv(DC_update)

array([[ 0.26007574, -0.26068285],
       [-0.26068285,  0.26143604]])

In [201]:
# NOTE - should we 'convert DC to complex'? i.e. divide by 2? 
def construct_Gamma_full_real_dc(DC_update, Gamma_update_complex, K, num_J_vars, invert=False):
    J = int((num_J_vars-1)/2)


    Gamma_full = np.zeros((K*num_J_vars, K*num_J_vars))
    if invert == True:
        DC_update = np.linalg.inv(DC_update)
    base_filt = np.zeros(num_J_vars)
    base_filt[0] = 1
    j_filt = np.tile(base_filt.astype(bool), K)

    for k in range(K):
        Gamma_full[j_filt,k*num_J_vars] = DC_update[:,k]



    for j in range(J):
        Gamma_n = Gamma_update_complex[j,:,:]
        if invert == True:
            Gamma_n = np.linalg.inv(Gamma_n)
        Gamma_n_real = reverse_rearrange_mat(transform_cov_c2r(Gamma_n),K)
        base_filt = np.zeros(num_J_vars)
        j_var = int(j*2 + 1)
        base_filt[j_var:j_var+2] = 1
        j_filt = np.tile(base_filt.astype(bool), K)
        # print(j_filt)
        for k in range(K):
            kj = int(k*2)
            Gamma_full[j_filt,k*num_J_vars+j_var:k*num_J_vars+j_var+2] = Gamma_n_real[:,kj:kj+2]

    return Gamma_full


In [166]:
def construct_Gamma_full_real(Gamma_update_complex, K, num_J_vars, invert=False):
    J = int(num_J_vars/2)
    Gamma_full = np.zeros((K*num_J_vars, K*num_J_vars))
    for j in range(J):
        Gamma_n = Gamma_update_complex[j,:,:]
        if invert == True:
            Gamma_n = np.linalg.inv(Gamma_n)
        Gamma_n_real = reverse_rearrange_mat(transform_cov_c2r(Gamma_n),K)
        base_filt = np.zeros(num_J_vars)
        j_var = int(j*2)
        base_filt[j_var:j_var+2] = 1
        j_filt = np.tile(base_filt.astype(bool), K)
        # print(j_filt)
        for k in range(K):
            kj = int(k*2)
            Gamma_full[j_filt,k*num_J_vars+j_var:k*num_J_vars+j_var+2] = Gamma_n_real[:,kj:kj+2]

    return Gamma_full


In [51]:
base_filt_dc = np.zeros(num_J_vars)
base_filt_dc[0] = 1
dc_filt = np.tile(base_filt_dc.astype(bool), K)
vec_dc = mus[l,:][dc_filt]

In [94]:
test = get_freq_vecs_real(mus[l,:], K, num_J_vars)

In [100]:
test[1]
# mus[l,:]

array([ 0.07191544,  0.11991725, -0.03485094, -0.20432273])

In [64]:
for jv in range(2, 5+2, 2):
    print(jv)

2
4
6


In [142]:
def get_freq_vecs_real_dc(vec, K, num_J_vars, dc=True):
    if dc is True:
        j_vecs = []
        base_filt = np.zeros(num_J_vars)
        base_filt_dc[0] = 1
        dc_filt = np.tile(base_filt_dc.astype(bool), K)
        vec_dc = vec[dc_filt]
        # vec_dc_expand = np.concatenate([np.array([vec_dc[k], 0]) for k in range(K)])
        j_vecs.append(vec_dc)
        for jv in range(1,num_J_vars,2):
            base_filt = np.zeros(num_J_vars)
            base_filt[jv:jv+2] = 1
            j_filt = np.tile(base_filt.astype(bool), K)
            vec_j = vec[j_filt]
            j_vecs.append(vec_j)
        return j_vecs
    else: 
        raise NotImplementedError

In [ ]:
def update_Gamma_complex(mus, Ups_invs, K, num_J_vars):
    '''
    mus is (trials x num_J_vars * K)
    Ups_inv is (trials x num_J_vars * K x num_J_vars * K)
    '''
    L = mus.shape[0]

    J = int(num_J_vars/2)
    mus_outer = np.zeros((L,J,K*2,K*2))
    Upss = np.zeros((L,J,K*2,K*2))
    
    for l in range(L):
        Ups_inv_j_vecs = get_freq_vecs_real(np.diag(Ups_invs[l,:,:]), K, num_J_vars)
        mu_js = get_freq_vecs_real(mus[l,:], K,num_J_vars)
        for j in range(J):
            mus_outer[l,j,:,:] = np.outer(mu_js[j], mu_js[j])
            Upss[l,j,:,:] = np.diag(1/Ups_inv_j_vecs[j])

    # enforce circulary symmetry
    k_mask_pre = 1 - np.eye(2)
    k_mask_inv = block_diag(*[k_mask_pre for k in range(K)])
    k_mask =  1 - k_mask_inv

    Gamma_update_complex = np.zeros((J,K,K), dtype=complex)
    for l in range(L):
        Sig_real = mus_outer[l,:,:,:]*k_mask + Upss[l,:,:,:]
        Sig_complex = np.zeros((J,K,K), dtype=complex)
        for j in range(J):
            Sig_complex[j,:,:] = est_cov_r2c(rearrange_mat(Sig_real[j,:,:],K))
        Gamma_update_complex += Sig_complex
    Gamma_update_complex = Gamma_update_complex / L

    return Gamma_update_complex